In [1]:
# 
# Module Zeno
# RLH 1/29/2021. updated 11/26/21. Added Lt, step transmission length 12/4/21.
#
module zeno

@doc """
    zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Lt=0.0, Ld=1.0, B=0.0)

Minimal model for growth in 1+1 dimensions.

Example: ```zenogrowth(200,400,200000)```
Create a mound with a fixed base layer 200 units long, with a maximum height of 400, calculated in 200k time steps. 
Return Hbot, Htop, L_array, TerraceLength, v.
"""
function zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Ld=1.0, B=0.0)   
    Profile_array = zeros(tmax,Hmax)
    
    diff = 0
    Hbot = 1
    Htop = 2
    sticking(L) = 1.0-B*((L^2)/(L+Ld)^2) # L-dependent sticking coefficient
    f(L,Ls,isTop) = (isTop ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 - Ls/(Ls+L))) # from upper terrace

    g(L,Ls,isBot) = (isBot ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 + Ls/(Ls+L))) # from lower terrace

    v = zeros(Hmax)
    L_array = zeros(Hmax)
    TerraceLength = zeros(Hmax)
    L_array[1] = Lmax


    for i in 1:tmax
        L_0 = copy(L_array)
        
        # Calculate the terrace lengths.
        TerraceLength = zeros(Hmax)
        for j in Hbot:Htop
            TerraceLength[j] = L_array[j] - L_array[j+1]
        end

        newHbot = 1
        for j in Hbot:Htop
            if L_array[j] >= Lmax
                newHbot = j
            end
        end
        Hbot = newHbot
    
        Htop_temp = copy(Htop)
        Hbot_temp = copy(Hbot)
    
    
        # Calculate the step velocities.
        # Changed Htop to Htop-1 to avoid nucleating early. RLH 11/25/21
        v = zeros(Hmax)
        for j in Hbot:Htop-1
            if L_array[j] >= Lmax      
                if L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
            end
            
            elseif L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
            end
        end
        
        trigger = 0 # trigger for Lmax correction
    
        for j in Hbot:Htop
            if L_array[j] >= Lc
                L_array[j+1] += v[j]
                if L_array[j+1] >= Lmax
                    
                    trigger = 1 # Lmax correction happened
                
                    L_array[j+1] = Lmax
                end
                if j==Htop 
                    Htop+=1 
                end
            end
        end
    
        if trigger == 1
        
            L_array = copy(L_0)  # return to previous step positiion
            flux_1 = 0
        
            # return to the old Hot and Hbot
            Htop = copy(Htop_temp)
            Hbot = copy(Hbot_temp)
        
            for j in Hbot:Htop
                if L_array[j] >= Lmax
                    t_length = Lmax - L_array[j+1] # calculate the terrace length of the Hbot step
                
                    # calculate a flux for the sub step 1 two make the step go to exactly Lmax
                    flux_1 =  t_length/a/(TerraceLength[j]*sticking(TerraceLength[j]) + TerraceLength[j+1]/2*sticking(TerraceLength[j+1])*(1-Ls/(Ls+TerraceLength[j+1])))
                end
            end
                            
            F_temp = copy(F) # save the default flux
            F = copy(flux_1) # replace the flux with flux_1 for sub step 1

        
            # recalculate the v_array for sub step 1
            v = zeros(Hmax)
            for j in Hbot:Htop-1
                if L_array[j] >= Lmax      
                    if L_array[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                end
            
                elseif L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                end
            end
                    
            #update the L array with sub step 1
            for j in Hbot:Htop
                if L_array[j] >= Lc
                    L_array[j+1] += v[j]
                    if j==Htop 
                        Htop+=1 
                    end
                end
            end
        
            # find correct Hbot for sub step 2
            newHbot = 1
            for j in Hbot:Htop
                if L_array[j] >= Lmax
                    newHbot = j
                end
            end
            Hbot = newHbot
        
            flux_2 = F_temp - flux_1

        
            # Calculate the terrace lengths again for sub step 2.
            TerraceLength = zeros(Hmax)
            for j in Hbot:Htop
                TerraceLength[j] = L_array[j] - L_array[j+1]
            end
                    
            F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
            # recalculate the v_array for sub step 2
            v = zeros(Hmax)
            for j in Hbot:Htop-1
                if L_array[j] >= Lmax      
                    if L_array[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                end
            
                elseif L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                end
            end
    
        
            #update the L array with sub step 2
            for j in Hbot:Htop
                if L_array[j] >= Lc
                    L_array[j+1] += v[j]
                    if j==Htop 
                        Htop+=1 
                    end
                end
            end
                  
            F = copy(F_temp) # return the flux to the default
    end
    Profile_array[i,:] = L_array
    end
    return Profile_array, Htop, Hbot

end

function create_profile(layer_lengths_array)
    xvals = []
    yvals = []
    mymaxheight = size(layer_lengths_array)[1]
    for int in 1:mymaxheight
        my_layer_length = layer_lengths_array[int]
        if(my_layer_length == 0)
            break
        end
        xvals = append!(xvals, [my_layer_length])
        yvals = append!(yvals, [int])
    end
    return xvals, yvals
end

function terrace_calc(x)
    terrace = []
    for i in 1:size(x)[1]-1
        terrace = append!(terrace, x[i]-x[i+1])
    end
    return terrace
end

function rms_calc(L_array)
    terrace = zeno.terrace_calc(L_array)
    mean = sum(terrace.*(1:length(terrace)))/sum(terrace)
    rms = sqrt(sum(terrace.*((1-mean:length(terrace)-mean).^2))/sum(terrace))
    return rms, mean
end

end # end module

Main.zeno

In [2]:
using Statistics;using PyPlot; pygui(true)

true

In [ ]:
# a = 1.0
# Hmax = 2500
# F = 0.05
# Lc = 0.5
# Ls = 0.015
# tmax = 35000
# Lmax = 10
# Ld = 0.1
# B = 0.00


# Profile_array, Htop, Hbot, = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=a, Ls=Ls, Lc=Lc, Ld= Ld, B=B)


In [ ]:
# x,y = zeno.create_profile(L_array)
# x = append!(x,0)
# y = append!(y,length(x))
# # println(x)

# println(sum(x)/Lmax)
# println((F*tmax+1))

# rms, mean = zeno.rms_calc(L_array)
# println(rms," ",mean)

# # figure()
# plot(x,y,"-o")

# plt.xlabel("width",fontsize=15)
# plt.ylabel("height",fontsize=15)

In [ ]:
# rms, mean = zeno.rms_calc(Profile_array[tmax,:])

In [ ]:
a = 1
Hmax = 2500
F = 0.05
Lc = 0.5
Ls = 0.015
tmax = 35000
# Lmax = 5
Ld = 0.1
B = 0.00

figure()

Lmax_array = vcat(1:10,20:10:100)

for i in 1:length(Lmax_array)
    
    Profile_array, Htop, Hbot = zeno.zenogrowth(Lmax_array[i], Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
    x,y = zeno.create_profile(Profile_array[tmax,:])
    x = append!(x,0)
    y = append!(y,length(x))
#     plot(x,y,markersize=6.0,marker="o",label="Lmax = $(Lmax_array[i])" )
    plot(x/Lmax_array[i],y,markersize=6.0,marker="o",label="Lmax = $(Lmax_array[i])" )
    
    rms, avg_h = zeno.rms_calc(Profile_array[tmax,:])
    println(avg_h)


    
end

# xlabel("Width (latice)",fontsize=15)
xlabel("Layer Covrage",fontsize=15)
ylabel("Height (layers)",fontsize=15)


legend()

tick_params(labelsize=15)
tight_layout()

In [ ]:
t = [1751.0,1751.0,1751.0,1751.0,1751.0,1751.0,1751.0,1750.9999999999995,1751.0,1751.0,1751.0000000000002,1751.0000000000002,
    1751.0000000000002,1751.0000000000002,1751.0000000000005,1751.0,1751.0000000000005,1751.0000000000002,1750.9999999999998]

figure()

plt.plot(Lmax_array,t,"-o",label = "actual thickness")


hlines(1751,1,100,"black","--",lw=2)
plt.text(30,1753,"expected average thickness",fontsize = 15)

ylabel("Thickness",fontsize=15)
xlabel("Lmax",fontsize=15)

# ylim(5100,5300)

legend(fontsize = 15)

tick_params(labelsize=15)
tight_layout()

In [ ]:
pygui(false)
# pygui(true)

In [ ]:
pygui(false)
# pygui(true)

In [ ]:
a = 1
Hmax = 50
F = 0.005
Lc = 0.5
Ls = 0.015
# tmax = 35000
Lmax = 1
Ld = 0.1
B = 0.00

t = [4500,4700,4900,5100,5300,6300,6500]


figure()

for i in 1:length(t)
    
    Profile_array, Htop, Hbot, diff = zeno.zenogrowth(Lmax, Hmax, t[i]; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)

    
    x,y = zeno.create_profile(Profile_array[t[i],:])
    x = append!(x,0)
    y = append!(y,length(x))
    plot(x,y,markersize=6.0,marker="o",label="t = $(t[i])")
    
    println(x)
    
#     rms, avg_h = zeno.rms_calc(Profile_array[t[i],:])
#     println(rms," ",avg_h," ",avg_h+diff/Lmax )
#     println(" ")

    
end

xlabel("Width (latice)",fontsize=15)
ylabel("Height (layers)",fontsize=15)

legend(fontsize=15,loc=1)

tick_params(labelsize=15)
tight_layout()

In [15]:
a = 1
Hmax = 1800
F = 0.01
Lc = 0.5
Ls = 0.015
tmax = 150000
Ld = 0.1
Lmax = 50


figure()

B_array = [0.0,0.01,0.02,0.03,0.04,0.05,0.07,0.10]

for i in 1:length(B_array)
    
    Profile_array, Htop, Hbot = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B_array[i])
    
    x,y = zeno.create_profile(Profile_array[tmax,:])
    x = append!(x,0)
    y = append!(y,length(x))
    plot(x,y,markersize=6.0,marker="o",label="B = $(B_array[i])" )
#     plot(x/Lmax,y,markersize=6.0,marker="o",label="B = $(B_array[i])" )
    
    rms, avg_h = zeno.rms_calc(Profile_array[tmax,:])
    println(avg_h)


    
end

xlabel("Width (latice)",fontsize=15)
# xlabel("Layer Covrage",fontsize=15)
ylabel("Height (layers)",fontsize=15)


legend()

tick_params(labelsize=15)
tight_layout()

1501.0
1487.2550273695365
1473.6169687515464
1460.0956082821635
1446.7054619257374
1433.4517138251051
1407.3791935740144
1369.4067293524733


0.9146577415540599

In [ ]:
pygui(true)

In [54]:
Lmax_array = vcat(1:0.2:3.4, 3.5:0.05:4.8,4.9:0.2:5.9, 6.0:0.5:20)

Lmax_array = vcat(1:0.2:3.0, 3.0:0.05:4.6, 4.8:0.2:5.8, 6.0:0.5:20)

Lmax_array = vcat(1:0.2:3.0, 3.0:0.05:5, 5.2:0.2:5.8, 6.0:0.5:8, 8:1:15, 17:2:40)

Lmax_array = vcat(1:0.2:3.6, 3.65:0.05:5.2, 5.4:0.2:5.8, 6.0:0.5:8, 8:1:15, 17:2:29, 30:5:80)

Lmax_array = vcat(1:0.2:4.0, 4.05:0.05:7, 7.5:0.5:10, 11:1:15, 17:2:29, 30:5:80)

Lmax_array = vcat(1:0.2:5, 5.05:0.05:8, 8.5:0.5:12, 13:1:20, 20:2:30, 35:5:50, 60:10:100)


# Lmax_array = vcat(4.9:0.1:5.5,5.6:0.2:7)
println(Lmax_array)

[1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.05, 5.1, 5.15, 5.2, 5.25, 5.3, 5.35, 5.4, 5.45, 5.5, 5.55, 5.6, 5.65, 5.7, 5.75, 5.8, 5.85, 5.9, 5.95, 6.0, 6.05, 6.1, 6.15, 6.2, 6.25, 6.3, 6.35, 6.4, 6.45, 6.5, 6.55, 6.6, 6.65, 6.7, 6.75, 6.8, 6.85, 6.9, 6.95, 7.0, 7.05, 7.1, 7.15, 7.2, 7.25, 7.3, 7.35, 7.4, 7.45, 7.5, 7.55, 7.6, 7.65, 7.7, 7.75, 7.8, 7.85, 7.9, 7.95, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]


In [20]:
a = 1
Hmax = 1800
F = 0.01
Lc = 0.5
Ls = 0.03
tmax = 150000
Ld = 0.1
B = 0.0

# Lmax_array = vcat(1:0.2:4, 4.1:0.1:5.5, 5.6:0.2:7, 7.5:0.5:10)

# Lmax_array = 1:0.2:3
# Lmax_array = vcat(3.2:0.2:4.4,4.5:0.1:4.8)
# Lmax_array = vcat(4.9:0.1:5.5,5.6:0.2:5.8,6.2:0.2:7)
# Lmax_array  = 7.5:0.5:10
# Lmax_array  = 10000:10000:100000

# Lmax_array = vcat(1:0.2:3.4, 3.5:0.05:4.8,4.9:0.2:5.9, 6.0:0.5:20)
# Lmax_array = vcat(1:0.2:3.0, 3.0:0.05:4.6, 4.8:0.2:5.8, 6.0:0.5:20)
# Lmax_array = vcat(1:0.2:5, 5.05:0.05:8, 8.5:0.5:12, 13:1:20, 20:2:30, 35:5:50, 60:10:100)


Lmax_array = 3:0.1:6
# Lmax_array = vcat(20:1:30, 32:2:50)

# Lmax_array = 40:5:70
# Lmax_array = 5.45:0.05:5.5



# Lmax_array = vcat(1:0.5:10.5)

# figure()
rough = []

for i in 1:length(Lmax_array)
    
    Profile_array, Htop, Hbot = zeno.zenogrowth(Lmax_array[i], Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
#     rms, avg_h = zeno.rms_calc(Profile_array[tmax,:])
#     println(rms," ",avg_h)
#     rms_f = []
#     h_f = []
#     for j in 1:100:tmax
#         rms, avg_h = zeno.rms_calc(Profile_array[j,:])
#         rms_f = append!(rms_f,rms)
#         h_f = append!(h_f,avg_h)
#     end
    
#     plot(h_f,rms_f,"-o",label="Lmax = $(Lmax_array[i])")
    
    t_layer = (1/F)

    rms_ini = []
#     for k in ceil(Int,(tmax - 520*t_layer)):ceil(Int,(tmax - 500*t_layer))
    for k in ceil(Int,(tmax - 520*t_layer)):ceil(Int,(tmax - 500*t_layer))
        rms, avg_h = zeno.rms_calc(Profile_array[k,:])
        rms_ini = append!(rms_ini,rms)
    end


    rms_final = []
    for k in ceil(Int,(tmax - 20*t_layer)):tmax
        rms, avg_h = zeno.rms_calc(Profile_array[k,:])
        rms_final = append!(rms_final,rms)
    end

    a = mean(rms_ini)
    b = mean(rms_final)
    c = (b-a)/a
    
#     println(Lmax_array[i])
#     println("mean:",a)
#     println("mean:",b)
#     println("diff:",c*100," %")
    println(c*100)
#     println("_")
    rough = append!(rough,c*100)  
    
end


# tick_params(labelsize=15)
# xlabel("Thickness (layers)",fontsize=15)
# ylabel("RMS roughness (layers)",fontsize=15)


# legend(fontsize=15)

# tick_params(labelsize=15)
# tight_layout()

0.003588790048551292
25.253552040206795
30.403614750696047
32.40664908026315


LoadError: DomainError with -28828.49620700849:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [21]:
pygui(false)

figure()

plot(Lmax_array,rough,"-o")

tight_layout()

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\maz_bass\.julia\packages\PyCall\ilqDX\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('x and y must have same first dimension, but have shapes (31,) and (4,)')
  File "C:\Users\maz_bass\.julia\conda\3\x86_64\lib\site-packages\matplotlib\pyplot.py", line 3578, in plot
    return gca().plot(
  File "C:\Users\maz_bass\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_axes.py", line 1721, in plot
    lines = [*self._get_lines(self, *args, data=data, **kwargs)]
  File "C:\Users\maz_bass\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_base.py", line 303, in __call__
    yield from self._plot_args(
  File "C:\Users\maz_bass\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_base.py", line 499, in _plot_args
    raise ValueError(f"x and y must have same first dimension, but "


In [18]:
rough[15]

0.001041869820671388

In [19]:
Lmax_array[15]

4.4

In [ ]:
Lmax_array[10]

In [87]:
a = 1
Hmax = 1800
F = 0.005
Lc = 0.5
Ls = 0.03
tmax = 300000
Ld = 0.1
B = 0.00

Lmax= 3.4


    
Profile_array, Htop, Hbot = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
# rms_f = []
# h_f = []
# for j in 1:10:tmax
#     rms, avg_h = zeno.rms_calc(Profile_array[j,:])
#     rms_f = append!(rms_f,rms)
#     h_f = append!(h_f,avg_h)
# end


# t_layer = (1/F)

# rms_ini = []
# for k in ceil(Int,(tmax - 1020*t_layer)):ceil(Int,(tmax - 1000*t_layer))
#     rms, avg_h = zeno.rms_calc(Profile_array[k,:])
#     rms_ini = append!(rms_ini,rms)
# end


# rms_final = []
# for k in ceil(Int,(tmax - 20*t_layer)):tmax
#     rms, avg_h = zeno.rms_calc(Profile_array[k,:])
#     rms_final = append!(rms_final,rms)
# end

# a = mean(rms_ini)
# b = mean(rms_final)
# c = (b-a)/a
    
# println("mean:",a)
# println("mean:",b)
# println("diff:",c*100," %")


([3.4 0.017 … 0.0 0.0; 3.4 0.034 … 0.0 0.0; … ; 3.4 3.4 … 0.0 0.0; 3.4 3.4 … 0.0 0.0], 1505, 502)

In [88]:
pygui(true)

x,y = zeno.create_profile(Profile_array[tmax,:])
x = append!(x,0)
y = append!(y,length(x))
# println(x)

println(sum(x)/Lmax)
println((F*tmax+1))

# rms, mean = zeno.rms_calc(Profile_array[tmax,:])
# println(rms," ",mean)

figure()
plot(x,y,"-o")

plt.xlabel("width",fontsize=15)
plt.ylabel("height",fontsize=15)

1503.0050835876616
1501.0


PyObject Text(29.222222222222214, 0.5, 'height')

In [ ]:
rms_f = []
h_f = []
for j in 1:10:tmax
    rms, avg_h = zeno.rms_calc(Profile_array[j,:])
    rms_f = append!(rms_f,rms)
    h_f = append!(h_f,avg_h)
end

In [ ]:
figure()

plot(h_f,rms_f,"-o",)

tick_params(labelsize=15)
xlabel("Thickness (layers)",fontsize=15)
ylabel("RMS roughness (layers)",fontsize=15)

# ylim(5100,5300)

legend(fontsize=15)

tick_params(labelsize=15)
tight_layout() 

In [95]:
a = 1
Hmax = 800
F = 0.05
Lc = 0.5
Ls = 0.03
tmax = 10000
Lmax = 3.4
Ld = 0.1
B = 0.00

0.0

In [96]:
diff = 0
Hbot = 1
Htop = 2
sticking(L) = 1.0-B*((L^2)/(L+Ld)^2) # L-dependent sticking coefficient
f(L,Ls,isTop) = (isTop ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 - Ls/(Ls+L))) # from upper terrace

g(L,Ls,isBot) = (isBot ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 + Ls/(Ls+L))) # from lower terrace

v = zeros(Hmax)
L_array = zeros(Hmax)
TerraceLength = zeros(Hmax)
L_array[1] = Lmax


for i in 1:tmax
    L_0 = copy(L_array)
#     println("L_0:",L_0)
    v = zeros(Hmax)

    # Calculate the terrace lengths.
    TerraceLength = zeros(Hmax)
    for j in Hbot:Htop
        TerraceLength[j] = L_array[j] - L_array[j+1]
    end


    newHbot = 1
    for j in Hbot:Htop
        if L_array[j] >= Lmax
            newHbot = j
        end
    end
    Hbot = newHbot
    
    Htop_temp = copy(Htop)
    Hbot_temp = copy(Hbot)
    
    #     println(Hbot," ",Htop)
    
    # Calculate the step velocities.
    # Changed Htop to Htop-1 to avoid nucleating early. RLH 11/25/21
    v = zeros(Hmax)
    for j in Hbot:Htop-1
        if L_array[j] >= Lmax      
            if L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
        end
            
        elseif L_array[j+1] < Lc
            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
        else
            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
        end
    end
        
    trigger = 0 # trigger for Lmax correction
    
    for j in Hbot:Htop
        if L_array[j] >= Lc
            L_array[j+1] += v[j]
            if L_array[j+1] >= Lmax
                    
                trigger = 1 # Lmax correction happened
                
                L_array[j+1] = Lmax
            end
            if j==Htop 
                Htop+=1 
            end
        end
    end
    
    if trigger == 1
#         println("yes")
        
        L_array = copy(L_0)  # return to previous step positiion
        flux_1 = 0
        
        # return to the old Hot and Hbot
        Htop = copy(Htop_temp)
        Hbot = copy(Hbot_temp)
        
        for j in Hbot:Htop
            if L_array[j] >= Lmax
                t_length = Lmax - L_array[j+1] # calculate the terrace length of the Hbot step
                
                # calculate a flux for the sub step 1 two make the step go to exactly Lmax
                flux_1 =  t_length/a/(TerraceLength[j]*sticking(TerraceLength[j]) + TerraceLength[j+1]/2*sticking(TerraceLength[j+1])*(1-Ls/(Ls+TerraceLength[j+1])))
            end
        end
                            
        F_temp = copy(F) # save the default flux
        F = copy(flux_1) # replace the flux with flux_1 for sub step 1
#         println("F1:",flux_1)

        
        # recalculate the v_array for sub step 1
        v = zeros(Hmax)
        for j in Hbot:Htop-1
            if L_array[j] >= Lmax      
                if L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
            end
            
            elseif L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
            end
        end
        
        
#         println("v1:",sum(v))
#         println("v1:",v)
                    
        #update the L array with sub step 1
        for j in Hbot:Htop
            if L_array[j] >= Lc
                L_array[j+1] += v[j]
                if j==Htop 
                    Htop+=1 
                end
            end
        end
        
        # find correct Hbot for sub step 2
        newHbot = 1
        for j in Hbot:Htop
            if L_array[j] >= Lmax
                newHbot = j
            end
        end
        Hbot = newHbot
        
        flux_2 = F_temp - flux_1
#         println("F2:",flux_2)
        
        # Calculate the terrace lengths again for sub step 2.
        TerraceLength = zeros(Hmax)
        for j in Hbot:Htop
            TerraceLength[j] = L_array[j] - L_array[j+1]
        end
                    
        F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
        # recalculate the v_array for sub step 2
        v = zeros(Hmax)
        for j in Hbot:Htop-1
            if L_array[j] >= Lmax      
                if L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
            end
            
            elseif L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
            end
        end
                                 
#         println("v2:",sum(v))
#         println("v2:",v)       
        
        #update the L array with sub step 2
        for j in Hbot:Htop
            if L_array[j] >= Lc
                L_array[j+1] += v[j]
                if j==Htop 
                    Htop+=1 
                end
            end
        end
                  
        F = copy(F_temp) # return the flux to the default
                

                
        
    end
    
    L_1 =  copy(L_array)
    L_diff = L_1 - L_0

#     println("L_diff:",sum(L_diff))
#     println("L_0",L_0)
#     println("L_1",L_1)
#     println("T:",TerraceLength)
#     println("v:",v)
#     println(Hbot," ",Htop," ",sum(v))
#     println(Hbot," ",Htop)
#     println(" ")
        
end

In [97]:
x,y = zeno.create_profile(L_array)
x = append!(x,0)
y = append!(y,length(x))
# println(x)

println(sum(x)/Lmax)
println((F*tmax+1))

rms, avg_h = zeno.rms_calc(L_array)
println(rms," ",avg_h)

figure()
plot(x,y,"-o")

plt.xlabel("width",fontsize=15)
plt.ylabel("height",fontsize=15)

501.0000000000038
501.0
1.4924641337608817 501.0000000000001


PyObject Text(38.097222222222214, 0.5, 'height')